# Parameter Shift and Finite Difference Gradients with Operator flow

In [1]:
import qiskit.aqua.operators as of
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import Parameter
import numpy as np
from qiskit import BasicAer

#### Build Expectation Measurement

In [2]:
ansatz_op = of.PrimitiveOp(RealAmplitudes(3, reps=3))
op_params = ansatz_op.primitive.ordered_parameters
qs = ansatz_op.num_qubits
expect_op = ~of.StateFn(of.Z^qs) @ ansatz_op @ of.Zero

# Uncomment to try with basis changes
# expect_op = ~of.StateFn((of.X^qs) + (of.Z^qs) + (of.Y^qs) + (of.I^qs)) @ ansatz_op @ of.Zero
# expect_op = of.PauliExpectation().convert(expect_op)

# Point at which we're taking the gradient
grad_point = ((np.pi/4)*np.ones(len(op_params))).tolist()
param_dict = dict(zip(op_params, grad_point))

print(expect_op)

ComposedOp(
[OperatorMeasurement(ZZZ),
CircuitStateFn(
     ┌──────────┐          ┌──────────┐                      ┌──────────┐»
q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├──────────────■────■──┤ RY(θ[6]) ├»
     ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐┌─┴─┐  │  └──────────┘»
q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├┤ X ├──┼───────■──────»
     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    »
q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├─────┤ X ├───┤ X ├────»
     └──────────┘     └───┘   └───┘    └──────────┘     └───┘   └───┘    »
«                           ┌──────────┐             
«q_0: ──────────────■────■──┤ RY(θ[9]) ├─────────────
«     ┌──────────┐┌─┴─┐  │  └──────────┘┌───────────┐
«q_1: ┤ RY(θ[7]) ├┤ X ├──┼───────■──────┤ RY(θ[10]) ├
«     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├───────────┤
«q_2: ┤ RY(θ[8]) ├─────┤ X ├───┤ X ├────┤ RY(θ[11]) ├
«     └──────────┘     └───┘   └───┘    └───────────┘
)])


#### Parameter Shift Gradients (Exact)

In [4]:
# Note: Doesn't handle any special cases
def param_shift_gradient(op_expr, params):
    grad_ops = []
    for param in params:
        grad_ops += [op_expr.bind_parameters({param: param + np.pi / 2}) -
                     op_expr.bind_parameters({param: param - np.pi / 2})]
    return .5 * of.ListOp(grad_ops)

# Note: this is just a one-liner!
ps_grad = .5 * of.ListOp([expect_op.bind_parameters({param: param + np.pi / 2}) - 
                          expect_op.bind_parameters({param: param - np.pi / 2}) 
                          for param in op_params])

In [5]:
ps_grad = param_shift_gradient(expect_op, op_params)
ps_grad_bound = ps_grad.bind_parameters(param_dict)
# print(ps_grad_bound)

ps_grad_vals = np.asarray(ps_grad_bound.eval())
print(ps_grad_vals)

[-0.55445752+0.j  0.08034587+0.j  0.2236517 +0.j -0.05981917+0.j
 -0.13840413+0.j  0.26784587+0.j -0.27542839+0.j  0.4736517 +0.j
 -0.12231917+0.j -0.24130704+0.j -0.30066626+0.j -0.38055217+0.j]


#### Finite (Forward and Central) Difference Gradients (Exact)

In [6]:
# Note: Central differences are more accurate but run 2p circuits, while forward differences only run p+1 circuits.
def finite_difference_gradient(op_expr, params, step=.01, center=True):
    if center:
        grad_ops = [op_expr.bind_parameters({param: param + (step/2)}) - 
                    op_expr.bind_parameters({param: param - (step/2)})
                    for param in params]
        return of.ListOp(grad_ops) / step
    else:
        grad_ops = [op_expr.bind_parameters({param: param + step}) for param in params]
        return (of.ListOp(grad_ops) - op_expr) / step

In [7]:
fd_grad = finite_difference_gradient(expect_op, op_params, step=np.pi/8, center=True)
fd_grad_bound = fd_grad.bind_parameters(param_dict)
# print(fd_grad_bound)
fd_grad_vals = np.asarray(fd_grad_bound.eval())
print(fd_grad_vals)

[-0.5509017 +0.j  0.0798306 +0.j  0.22221738+0.j -0.05943554+0.j
 -0.13751652+0.j  0.26612813+0.j -0.27366203+0.j  0.4706141 +0.j
 -0.12153472+0.j -0.23975951+0.j -0.29873804+0.j -0.37811164+0.j]


In [8]:
# Approximation Error
grad_diffs = ps_grad_vals - fd_grad_vals
print(grad_diffs)
np.mean(np.abs(grad_diffs))

[-0.00355582+0.j  0.00051527+0.j  0.00143431+0.j -0.00038363+0.j
 -0.00088761+0.j  0.00171774+0.j -0.00176636+0.j  0.0030376 +0.j
 -0.00078445+0.j -0.00154754+0.j -0.00192822+0.j -0.00244054+0.j]


0.0016665898013804355

#### Circuit Sampling Instead of Exact Evaluation

In [9]:
# Trying with sampling
cs = of.CircuitSampler(BasicAer.get_backend('qasm_simulator'))
cs.quantum_instance.run_config.shots = 10000
sampled_ps_op = cs.convert(ps_grad_bound)
sampled_fd_op = cs.convert(fd_grad_bound)
sampled_ps_vals = np.asarray(sampled_ps_op.eval())
sampled_fd_vals = np.asarray(sampled_fd_op.eval())
print(sampled_ps_vals)
print(sampled_fd_vals)

[-0.02325+0.j -0.3575 +0.j  0.0727 +0.j -0.03755+0.j -0.2823 +0.j
 -0.16725+0.j -0.0718 +0.j -0.18805+0.j -0.103  +0.j  0.00385+0.j
  0.0068 +0.j  0.00365+0.j]
[-7.25881171+0.j -7.48447425+0.j -7.28345302+0.j -7.34959549+0.j
 -7.27956229+0.j -7.34829858+0.j -7.27956229+0.j -7.41055031+0.j
 -7.27826538+0.j -7.2341704 +0.j -7.29642213+0.j -7.23027966+0.j]


In [10]:
# Approximation Error, Sampled PS
sampled_ps_grad_diff = ps_grad_vals - sampled_ps_vals
print(sampled_ps_grad_diff)
np.mean(np.abs(sampled_ps_grad_diff))

[-0.53120752+0.j  0.43784587+0.j  0.1509517 +0.j -0.02226917+0.j
  0.14389587+0.j  0.43509587+0.j -0.20362839+0.j  0.6617017 +0.j
 -0.01931917+0.j -0.24515704+0.j -0.30746626+0.j -0.38420217+0.j]


0.2952283945979229

In [11]:
# Approximation Error, Sampled FD (compare to PS unsampled, which is exact)
sampled_fd_grad_diff = ps_grad_vals - sampled_fd_vals
print(sampled_fd_grad_diff)
np.mean(np.abs(sampled_fd_grad_diff))

[6.70435419+0.j 7.56482012+0.j 7.50710472+0.j 7.28977632+0.j
 7.14115816+0.j 7.61614445+0.j 7.0041339 +0.j 7.88420201+0.j
 7.1559462 +0.j 6.99286335+0.j 6.99575587+0.j 6.84972749+0.j]


7.225498898167345